In [4]:
import pandas as pd
import numpy as np

from collections import Counter

In [5]:
df = pd.read_csv('data/data.csv')
indicators = pd.read_csv('data/indicators.csv')
print(df.shape)
df = df.merge(indicators, on='indicatorId', how='left')
print(df.shape)

(36592, 6)
(36592, 16)


In [6]:
df = df[[ 'geoUnit', 'year', 'value', 'name', 'theme', 'disaggregations']]

In [7]:
language = pd.read_json('data/all.json')

In [8]:
language = language[['name', 'cca3', 'languages']]

In [9]:
df = df.merge(language, how='inner', left_on='geoUnit', right_on='cca3')

In [10]:
edu_map = {
    'Enrolment in primary education, female (number)': 'Primary education',
    'Enrolment in secondary education, female (number)': 'Secondary education',
    'Enrolment in primary education, male (number)': 'Primary education',
    'Enrolment in secondary education, male (number)': 'Secondary education',
}

m_f_map = {
    'Enrolment in primary education, female (number)': 'f',
    'Enrolment in secondary education, female (number)': 'f',
    'Enrolment in primary education, male (number)': 'm',
    'Enrolment in secondary education, male (number)': 'm',
}

In [11]:
df['Eduaction Level Enrolment'] = df.name_x.map(edu_map)
df['Gender'] = df.name_x.map(m_f_map)

In [12]:
df['Country'] = df.apply(lambda x: x['name_y']['common'], axis=1)

In [13]:
df['Languages'] = df.apply(lambda x: x['languages'].values(), axis=1)

In [14]:
df = df[['geoUnit', 'year', 'value', 'Eduaction Level Enrolment', 'Gender', 'Country', 'Languages']]

In [15]:
# Convert dict_values to strings if necessary
df["Languages"] = df.apply(lambda x: list(x["Languages"]), axis=1)

df['num_lang'] = df.apply(lambda x: len(x['Languages']), axis=1)

df['main_language'] = df.apply(lambda x: x["Languages"][0], axis=1)

In [16]:
df_expl = df.explode('Languages')


df_expl['max_year'] = df_expl.groupby('Country')['year'].transform('max')

In [17]:
mask1 = (df_expl.year == df_expl.max_year) & (df_expl.year>=2022)
df1 = df_expl[mask1].groupby("Languages")[["value"]].sum().reset_index().rename(columns={'value': 'last_year'})

In [18]:
mask2 = (df_expl.year == df_expl.max_year-1) & (df_expl.year>=2021)
df2 = df_expl[mask2].groupby("Languages")[["value"]].sum().reset_index().rename(columns={'value': 'prev_year'})

In [19]:
df1 = df1.merge(df2, on='Languages', how='left')

In [20]:
df1['YoY (%)'] = (df1.last_year/df1.prev_year-1)*100

In [21]:
df1.sort_values('YoY (%)', ascending=False).head(20)

,Languages,last_year,prev_year,YoY (%)
106,Swahili,32574494.0,10554126.0,208.642269
86,Papiamento,41436.0,17215.0,140.697067
80,Niuean,410.0,222.0,84.684685
32,French,91603117.0,61351450.0,49.308805
12,Bislama,113261.0,92189.0,22.857391
3,Arabic,59591144.0,54236642.0,9.872481
85,Palauan,2799.0,2571.0,8.868145
105,Spanish,89248311.0,83051327.0,7.461632
68,Maldivian,78619.0,73602.0,6.816391
122,Turkmen,1493441.0,1413687.0,5.641560


In [22]:
df1.sum()

Languages    AfrikaansAlbanianAmharicArabicArmenianAymaraAz...
last_year                                    2217181379.841848
prev_year                                    2104625809.162801
YoY (%)                                             501.251758
dtype: object

In [25]:
df_expl.to_excel('primary_secondary_enrolment_w_language.xlsx')